<a href="https://colab.research.google.com/github/stephenbeckr/convex-optimization-class/blob/master/Demos/RPCA_case_study_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case study: RPCA

In-class challenge: how many different algorithms could you use to solve the RPCA problem?

APPM 5630 Adv Convex Optimization, Spring 2023, Professor Stephen Becker

In [ ]:
import numpy as np
from scipy import linalg
from numpy.linalg import norm
from numpy.random import default_rng
import cvxpy as cvx
%reload_ext autoreload
%autoreload 2
!wget -nv 'https://github.com/stephenbeckr/convex-optimization-class/raw/master/utilities/firstOrderMethods.py'
!wget -nv 'https://github.com/stephenbeckr/convex-optimization-class/raw/master/utilities/secondOrderMethods.py'
from firstOrderMethods import gradientDescent, lassoSolver, createTestProblem, runAllTestProblems, DouglasRachford, bookkeeper
from secondOrderMethods import NewtonsMethod

## The Robust PCA (RPCA) problem
$$\min_{L,S} \;\frac12\|L+S-Y\|_F^2 + \lambda_L\|L\|_* + \lambda_S\|S\|_1$$
where $L,S$ are matrices of the same size; $Y$ is the target matrix that we wish to approximately decompose, $Y \approx L + S$, into a low-rank part $L$ and a sparse part $S$

In [ ]:
rng = default_rng(1)
m,n = 7,8

# for use later
def vec(L,S):
    return np.concatenate( (L.ravel(), S.ravel() ))
def mat( LS ):
    L = LS[:m*n].reshape( (m,n ))
    S = LS[m*n:].reshape( (m,n ))
    return L, S

# Generate a "signal". Need not be solution to optimization problem
rr  = 4
LL  = rng.standard_normal( (m,rr) ) @ rng.standard_normal( (rr,n) ) # low-rank
SS  = rng.standard_normal( (m,n) )
SS[ np.abs(SS) < np.quantile( np.abs(SS), .9 ) ] = 0  # sparse
Y   = LL + SS + .01*rng.standard_normal( (m,n) ) # Noisy observations

# with np.printoptions(precision=3, suppress=True):
#     print(SS)

### Solve in CVXPY for reference solution

This is one way to solve: the $\ell_1$ and nuclear norms can be written in terms of epigraphs of standard cones and such, and then solved via standard methods (IPM, ADMM)

In [ ]:
# Solve in cvxpy
L     = cvx.Variable( (m,n) )
S     = cvx.Variable( (m,n) )
lambdaL = 3e-3
lambdaS = 1.7e-3

def prox_l1( X, t, lambdaS ):
    return np.sign(X)*np.maximum( 0, np.fabs(X) - lambdaS*t )

def prox_nuclearNorm( X, t, lambdaL ):
   U, s, Vh = linalg.svd(X, full_matrices=False )
   s = prox_l1( s, t, lambdaL )
   return U@( s.reshape((-1,1))*Vh )

def objective_vec( LSvec ):
    L,S = mat(LSvec)
    resid = norm( L+S - Y)
    return resid**2/2 + lambdaL*np.sum( linalg.svdvals(L) ) + lambdaS*norm( S.ravel(), ord=1)
obj   = cvx.Minimize( cvx.sum_squares(L+S-Y)/2 + lambdaL*cvx.norm(L, "nuc") + lambdaS*cvx.pnorm(S,p=1) )
prob  = cvx.Problem(obj)
highPrecision = {'solver':cvx.CVXOPT,'max_iters':400,'abstol':1e-11,'reltol':1e-11}
prob.solve(**highPrecision)
L_CVX = L.value 
S_CVX = S.value 
fTrue = prob.value

with np.printoptions(precision=3, suppress=True):
    print( linalg.svdvals( L_CVX ))
    print( linalg.svdvals( LL ) )
    print(SS)
    print(S_CVX)

[10.374  4.849  2.435  1.788  0.466  0.314  0.   ]
[10.373  4.687  2.211  1.507  0.     0.     0.   ]
[[ 0.     0.     0.     0.     0.     0.     0.     0.   ]
 [-1.649  0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.    -1.482  0.     0.     0.     0.    -1.631  0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.    -2.251  0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.    -1.514]
 [ 1.753  0.     0.     0.     0.     0.     0.     0.   ]]
[[-0.    -0.    -0.     0.    -0.     0.     0.    -0.   ]
 [-0.54   0.     0.     0.     0.     0.    -0.     0.   ]
 [ 0.    -0.61  -0.     0.    -0.     0.    -0.666 -0.   ]
 [-0.    -0.    -0.    -0.     0.    -0.671 -0.    -0.   ]
 [ 0.     0.    -0.    -1.694 -0.     0.    -0.     0.   ]
 [ 0.     0.    -0.    -0.     0.     0.    -0.    -0.912]
 [ 0.     0.    -0.188  0.     0.    -0.    -0.     0.   ]]


# The algorithms

### Proximal gradient descent (or FISTA)

In [ ]:
def prox( LSvec, t ):
    L,S = mat(LSvec)
    L   = prox_nuclearNorm( L, t, lambdaL )
    S   = prox_l1( S, t, lambdaS)
    return vec( L, S )
def prox_obj( LSvec ):
    L,S = mat(LSvec)
    return lambdaL*np.sum( linalg.svdvals(L) ) + lambdaS*norm( S.ravel(), ord=1)

def f( LSvec ):
    L,S = mat(LSvec)
    resid = norm( L+S - Y)
    return resid**2/2

def grad( LSvec ):
    L,S = mat(LSvec)
    res = L+S-Y
    return np.concatenate( ( res.ravel(), res.ravel() ) )
    
print( fTrue )
refSoln = vec( L_CVX, S_CVX)
errFcn  = lambda LS : norm( LS-refSoln )/norm(refSoln)
LSvec0 = np.zeros(2*m*n)  # starting point
LS, data = gradientDescent( f, grad, LSvec0, prox, prox_obj, stepsize = 0.5, tol=1e-16, maxIters=1e4,
    errorFunction=errFcn, ArmijoLinesearch=False, acceleration=True, restart=500, saveHistory=True )
L,S = mat(LS)
# with np.printoptions(precision=3, suppress=True):
#     print( L )
#     print( S )

0.06968472982246328
Iter.  Objective Stepsize  Error
-----  --------- --------  -------
    0  9.38e-02  5.00e-01  6.79e-01
  500  6.97e-02  5.00e-01  2.04e-02
 1000  6.97e-02  5.00e-01  1.96e-03
 1500  6.97e-02  5.00e-01  2.45e-04
 2000  6.97e-02  5.00e-01  3.11e-05
 2500  6.97e-02  5.00e-01  3.94e-06
 2873  6.97e-02  5.00e-01  3.35e-07
Iter 2873 Quitting due to stagnating objective value


### Variable Projection  (VarPro) + proximal gradient descent
$$\min_{L} \lambda_L\|L\|_* + \underbrace{\min_S \frac12\|L+S-Y\|_F^2+\lambda_S\|S\|_1}_{f(L)}$$
where
$$\nabla f(L) = L+S-Y\quad\text{where }S\text{ solves the partial minimization problem}
$$

and of course you could also switch this around and flip the roles of $S$ and $L$

In [ ]:
def S_from_L( L ):
    return prox_l1( Y-L, 1, lambdaS )

def prox( Lvec, t ):
    L   = Lvec.reshape( (m,n) )
    L   = prox_nuclearNorm( L, t, lambdaL )
    return L.ravel()
    
def prox_obj( Lvec ):
    L   = Lvec.reshape( (m,n) )
    return lambdaL*np.sum( linalg.svdvals(L) )

def f( Lvec ):
    L   = Lvec.reshape( (m,n) )
    S   = S_from_L(L)
    resid = norm( L+S - Y)
    return resid**2/2 + lambdaS*norm(S.ravel(),ord=1)

def grad( Lvec ):
    L   = Lvec.reshape( (m,n) )
    S   = S_from_L(L)
    res = L+S-Y
    return res.ravel()
    
print( fTrue )
refSoln = vec( L_CVX, S_CVX)
def errFcnL( Lvec ):
    L   = Lvec.reshape( (m,n) )
    S   = S_from_L(L)
    return norm( vec(L,S)-refSoln )/norm(refSoln)

LSvec0 = np.zeros(2*m*n)  # starting point
Lvec, data = gradientDescent( f, grad, np.zeros(m*n), prox, prox_obj, stepsize = 0.5, tol=1e-16, maxIters=1e4,
    errorFunction=errFcnL, ArmijoLinesearch=False, acceleration=True, restart=250, saveHistory=True )
    # TODO: if restart < 0, set saveHistory = True
    # NOTE: I fiddled with fNew, make sure that still passes unit tests. Should do better?
L   = Lvec.reshape( (m,n) )
S   = S_from_L(L)

0.06968472982246328
Iter.  Objective Stepsize  Error
-----  --------- --------  -------
    0  1.15e-01  5.00e-01  1.39e+00
  500  6.98e-02  5.00e-01  5.58e-02
 1000  6.97e-02  5.00e-01  7.36e-04
 1500  6.97e-02  5.00e-01  8.33e-06
 1757  6.97e-02  5.00e-01  6.29e-07
Iter 1757 Quitting due to stagnating objective value


### Alternating minimization

In [ ]:
L   = Y
maxIters = int(3e4)
printEvery = int( maxIters/20 )
print("Iter.  Error")
print("-----  -------")
for k in range(maxIters):
    S   = prox_l1( Y-L, 1, lambdaS )
    L   = prox_nuclearNorm( Y-S, 1, lambdaL )
    err = errFcn( vec(L,S) )
    if not k % printEvery :
        print(f"{k:5d}  {err:.2e}")

Iter.  Error
-----  -------
    0  2.69e-01
 1500  1.08e-01
 3000  4.76e-02
 4500  2.62e-02
 6000  1.45e-02
 7500  7.96e-03
 9000  4.33e-03
10500  2.35e-03
12000  1.27e-03
13500  6.86e-04
15000  3.70e-04
16500  2.00e-04
18000  1.08e-04
19500  5.81e-05
21000  3.13e-05
22500  1.69e-05
24000  9.07e-06
25500  4.84e-06
27000  2.56e-06
28500  1.33e-06


### Douglas-Rachford

In [ ]:
def prox1( LSvec, t ):
    L,S = mat(LSvec)
    L   = prox_nuclearNorm( L, t, lambdaL )
    S   = prox_l1( S, t, lambdaS)
    return vec( L, S )

def prox2( LSvec, t ):
    L,S = mat(LSvec)
    L_new   = -t*(S - (1+t)/t*L - Y)/(1+2*t)
    S_new   = -t*(L - (1+t)/t*S - Y)/(1+2*t)
    return vec( L_new, S_new )

LSvec0 = np.zeros(2*m*n)
LS, data = DouglasRachford( prox1, prox2, LSvec0, gamma=4e1, F=objective_vec, overrelax = 1.999, 
    tol =1e-10, maxIters = 1e3, errorFunction=errFcn, printEvery=None )

L,S = mat(LS)

Iter.  Objective Error
-----  --------- -------
    0  1.05e-01  6.79e-01
   50  7.20e-02  2.88e-01
  100  7.01e-02  1.51e-01
  150  6.98e-02  8.47e-02
  200  6.97e-02  4.31e-02
  250  6.97e-02  2.06e-02
  300  6.97e-02  9.88e-03
  350  6.97e-02  4.73e-03
  400  6.97e-02  2.26e-03
  450  6.97e-02  1.09e-03
  500  6.97e-02  5.22e-04
  550  6.97e-02  2.51e-04
  600  6.97e-02  1.21e-04
  650  6.97e-02  5.82e-05
  700  6.97e-02  2.80e-05
  750  6.97e-02  1.35e-05
  800  6.97e-02  6.41e-06
  850  6.97e-02  3.00e-06
  900  6.97e-02  1.36e-06
  936  6.97e-02  7.39e-07
==  Quitting due to stagnating objective value  ==


### Primal-dual method
(Chambolle-Pick / Vu / Condat)

TBD

### Burer-Monteiro splitting on $\|L\|_*$ term

TBD

Exploit the fact that $\|L\|_* = \min_{U,V}\; \frac12(\|U\|_F^2 + \|V\|_F^2) \quad\text{s.t.}UV^T = L$ and make a change of variables

cf. [Adapting Regularized Low Rank Models for Parallel Architectures](https://doi.org/10.1137/17M1147342) Derek Driggs, Stephen Becker, Aleksandr Aravkin (SISC, 2019)